In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import datasets
import utils
import shutil
import os

In [2]:
data = datasets.TDSC()
print(len(data))

100


In [3]:
root_path = "./data/tdsc/tumors"

In [9]:
def find_tumors(mask: np) -> np:
    '''
    Finds all bounding boxes in volume containing tumors
    
    parameters:
    - mask: numpy array with shape (x,y,d)
    
    returns:
    - numpy array with (None, [x,y,w,h,d])
    '''
    
    _, _, depth = mask.shape
    bbxs = []
    for d in range(depth):
        sli = mask[:,:,d]
        if sli.max() != 0:
            *bbx, = utils.image.find_bbx(sli)
            bbx.append(d)
            bbxs += [bbx]
    return np.array(bbxs)

In [25]:
for idx, sample in tqdm(enumerate(data)):
    v,m,l = sample
    m = m*255
    bbxs = find_tumors(m)
    for bbx in bbxs:
        y,x,h,w,d = bbx
        img = v[x:x+w, y:y+h,d]
        msk = m[x:x+w, y:y+h,d]
        os.makedirs(f"{root_path}/data/{l}/{idx}", exist_ok=True)
        os.makedirs(f"{root_path}/mask/{l}/{idx}", exist_ok=True)
        cv2.imwrite(f"{root_path}/data/{l}/{idx}/{d}.png", img)
        cv2.imwrite(f"{root_path}/mask/{l}/{idx}/{d}.png", msk)

0it [00:00, ?it/s]

In [13]:
def fun(*numbers, **options):
    print(numbers, options)
    type(options)
    

fun('10', 'ali', 21, 'jakesh', ali=10, text="hassan", flag=True)

a = [1,2,2,3,4,5]
b = {"ali" : 10}
print(*a)

('10', 'ali', 21, 'jakesh') {'ali': 10, 'text': 'hassan', 'flag': True}
1 2 2 3 4 5
